# Chatbot Educativo Interactivo
Proyecto adaptado con TF-IDF, Naive Bayes, similitud de coseno y ventana gráfica (Tkinter)

## 1. Importación de Librerías

In [2]:

import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import spacy
import tkinter as tk
from tkinter import ttk
import threading
import time

import unicodedata
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import make_pipeline

nlp = spacy.load("es_core_news_md")

## 2. Definición del Dataset

In [3]:
## 📥 Cargar el Dataset
chatbot_df = pd.read_csv(r"C:\Users\darly\Downloads\dataEducacion_final_393_agregado.csv")

chatbot_df.head()

,consulta,categoria,respuesta,respuesta_compuesta,respuestas_compuestas
0,¿Cómo me inscribo al próximo semestre?,matriculas,"Para inscribirte al próximo semestre, debes in...","Para me inscribo al próximo semestre, Para ins...","Para inscribirte inscribo al próximo semestre,..."
1,¿Cuál es la fecha límite para inscribirse?,matriculas,La fecha límite para completar tu inscripción ...,"La la fecha límite para inscribirse es, La fec...","La la fecha límite para inscribirse es, La fec..."
2,¿Qué documentos necesito para la matrícula?,matriculas,Para realizar tu matrícula necesitas: identifi...,Lo que necesitas saber sobre documentos necesi...,Lo que necesitas saber sobre documentos necesi...
3,¿Cuánto cuesta la inscripción?,matriculas,El costo de inscripción para el semestre actua...,El valor relacionado con cuesta la inscripción...,El valor relacionado con cuesta la inscripción...
4,¿Dónde puedo pagar mi matrícula?,matriculas,Puedes pagar tu matrícula en cualquier sucursa...,"Puedes hacerlo en el lugar donde, Puedes pagar...","Puedes realizarlo en el lugar donde, Puedes pa..."


## 3. Preprocesamiento y limpieza 

In [4]:
#limpieza de los datos #Incluye:
#- Minúsculas
#- Eliminación de tildes
#- Lematización
#- Conserva palabras importantes como "dónde", "cuándo", "cómo", "qué", etc.


# Palabras que queremos conservar (normalizadas)
custom_stopwords_to_keep = {"no", "si", "donde", "cuando", "como", "que", "cual", "cuanto"}

def normalize_text(text):
    # Paso 1: convertir a minúsculas y eliminar tildes
    text = str(text).lower()
    text = ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )
    text = re.sub(r"[^a-zA-Züñ¿?¡! ]", "", text)  # conservamos solo letras simples

    # Paso 2: procesar con Spacy
    doc = nlp(text)
    
    # Paso 3: eliminar stopwords solo si no están en custom_stopwords_to_keep
    tokens = [
        token.lemma_ for token in doc
        if not (token.is_stop and token.lemma_.lower() not in custom_stopwords_to_keep)
    ]
    
    return " ".join(tokens)


In [5]:
# Aplicar limpieza
chatbot_df["consulta_limpia"] = chatbot_df["consulta"].apply(normalize_text)

In [6]:
#Vamos a crear un modelo de clasificación de intención por categoría usando n-gramas de 1 a 3.

X = chatbot_df["consulta_limpia"]
y = chatbot_df["categoria"]

## 4. Vectorización TF-IDF

In [7]:
#partir set 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
#crear estructura de entranamiento y medir la clasificacion

model = make_pipeline(
    TfidfVectorizer(ngram_range=(1, 2),# Usa un contexto de hasta 3 palabras
                    min_df=1),           # Aparece en al menos 2 consulta
    MultinomialNB()
)


## 4. Entrenamiento del Modelo Naive Bayes

In [9]:
#entrenar modelo
model.fit(X_train, y_train)
print("Precisión del modelo:", model.score(X_test, y_test))
preds = model.predict(X_test)
print(classification_report(y_test, preds))


Precisión del modelo: 0.8734177215189873
                precision    recall  f1-score   support

calificaciones       0.87      0.93      0.90        14
      horarios       0.82      1.00      0.90        14
 instalaciones       1.00      0.92      0.96        12
    matriculas       0.83      0.91      0.87        11
    requisitos       0.83      0.71      0.77        14
      tramites       0.92      0.79      0.85        14

      accuracy                           0.87        79
     macro avg       0.88      0.88      0.87        79
  weighted avg       0.88      0.87      0.87        79



## 5. Función para Predecir y Responder con función de similitud

In [18]:
## 💬 Motor de Chatbot
#Definimos saludos y despedidas y usamos clasificación + similitud para elegir la mejor respuesta en cada categoría.

saludos = ["hola", "buenos dias", "buenas tardes", "buenas noches", "hola que tal", "como estas", "ey"]
respuestas_saludo = ["¡Hola! ¿En qué puedo ayudarte?", "¡Buenos días! ¿Qué necesitas?", "Hola, dime tu duda."]

despedidas = ["gracias", "hasta luego", "adios", "nos vemos", "bye", "chao"]
respuestas_despedida = ["¡Hasta pronto!", "Gracias por tu consulta. ¡Éxitos!", "Nos vemos."]

def responder(pregunta):
    pregunta_limpia = normalize_text(pregunta)
    print("🔍 Consulta limpia →", pregunta_limpia)
    
    if any(saludo in pregunta.lower() for saludo in saludos):
        return "saludo",random.choice(respuestas_saludo)
    
    if any(despedida in pregunta.lower() for despedida in despedidas):
        return "despedida", random.choice(respuestas_despedida)
    
    # 1. Clasificar la intención (etiqueta)
    etiqueta = model.predict([pregunta_limpia])[0]
    print("📌 Categoría detectada →", etiqueta)
    
    # 2. Filtrar respuestas de esa categoría
    respuestas_categoria = chatbot_df[chatbot_df["categoria"] == etiqueta].copy()

    # 3. Normalizar las respuestas para compararlas
    respuestas_categoria["respuesta_limpia"] = respuestas_categoria["respuestas_compuestas"].apply(normalize_text)

    # 4. Vectorizar respuestas limpias
    vectorizer = model.named_steps["tfidfvectorizer"]
    respuestas_categoria["respuesta_vec"] = respuestas_categoria["respuesta_limpia"].apply(lambda x: vectorizer.transform([x]))
    
    pregunta_vec = vectorizer.transform([pregunta_limpia])
    
    # 5. Calcular similitud contra respuestas
    respuestas_categoria["similitud"] = respuestas_categoria["respuesta_vec"].apply(lambda x: cosine_similarity(x, pregunta_vec)[0][0])
    
    # 6. Mostrar top 5 respuestas más similares
    print("\n🎯 Top 5 respuestas más similares:")
    top5 = respuestas_categoria[["respuestas_compuestas", "similitud"]].sort_values(by="similitud", ascending=False).head(5)
    print(top5.to_string(index=False))

    # 7. Devolver la mejor respuesta
    mejor_idx = respuestas_categoria["similitud"].idxmax()
    
    return etiqueta,chatbot_df.loc[mejor_idx, "respuesta"]


## 6. Creación de la Interfaz Gráfica

In [23]:
# Crear ventana principal
ventana = tk.Tk()
ventana.title("Chat Educativo")
ventana.geometry("550x700")

# Frame de conversación
frame_conversacion = tk.Frame(ventana, bd=2, relief=tk.GROOVE)
frame_conversacion.pack(padx=15, pady=(15,5), fill=tk.BOTH, expand=True)

# Scrollbar y área de texto
scrollbar = tk.Scrollbar(frame_conversacion)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

texto_conversacion = tk.Text(frame_conversacion, wrap=tk.WORD, yscrollcommand=scrollbar.set, font=("Arial", 12))
texto_conversacion.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
scrollbar.config(command=texto_conversacion.yview)

# Configurar colores (tags)
texto_conversacion.tag_configure("usuario", foreground="black")
texto_conversacion.tag_configure("bot", foreground="blue")

# Barra de progreso
progress = ttk.Progressbar(ventana, mode='indeterminate')
progress.pack(padx=15, pady=(5,5), fill=tk.X)
progress.pack_forget()

# Frame de entrada
frame_entrada = tk.Frame(ventana)
frame_entrada.pack(padx=15, pady=(10,15), fill=tk.X)

# Entrada de texto
entrada_texto = tk.Entry(frame_entrada, font=("Arial", 14))
entrada_texto.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=(0,10))

# Botón de enviar
boton_enviar = tk.Button(frame_entrada, text="Enviar", font=("Arial", 12, "bold"), command=lambda: enviar_pregunta())
boton_enviar.pack(side=tk.RIGHT)

# Función para enviar pregunta
def enviar_pregunta():
    pregunta = entrada_texto.get()
    if pregunta.strip() == "":
        return
    texto_conversacion.insert(tk.END, f"👤 Tú: {pregunta}\n\n", "usuario")  # Se aplica el estilo "usuario"
    texto_conversacion.see(tk.END)
    entrada_texto.delete(0, tk.END)
    progress.pack(padx=15, pady=(5,5), fill=tk.X)
    progress.start()
    threading.Thread(target=procesar_pregunta, args=(pregunta,)).start()

# Función para procesar la respuesta
def procesar_pregunta(pregunta_usuario):
    time.sleep(1)
    categoria_predicha, respuesta = responder(pregunta_usuario)
    progress.stop()
    progress.pack_forget()
    texto_conversacion.insert(tk.END, f"🤖 Bot [{categoria_predicha}]: {respuesta}\n\n", "bot")  # Se aplica el estilo "bot"
    texto_conversacion.see(tk.END)

# Activar evento de Enter para enviar
ventana.bind('<Return>', lambda event: enviar_pregunta())

# Iniciar la ventana
ventana.mainloop()


🔍 Consulta limpia → hola
🔍 Consulta limpia → ey
🔍 Consulta limpia → donde quedar biblioteca
📌 Categoría detectada → instalaciones

🎯 Top 5 respuestas más similares:
                                                                                                                                                                                                                                                                            respuestas_compuestas  similitud
                                                                                                                                                                Puedes realizarlo en el lugar donde, Hay estaciones de carga en la biblioteca, cafetería y algunos pasillos centrales del campus.   0.493172
                                                                                                                                                             Puedes realizarlo en el lugar donde, En las máquinas de autoservicio ubicada